In [95]:
# TODO: make a proper module and remove this hack
import os
import sys
sys.path.append("/home/robbe/open-hardware-definitions/")

from parsers.common import DATASHEET_DIR

from py_pdf_parser import tables
from py_pdf_parser.common import BoundingBox
from py_pdf_parser.loaders import load_file

from pprint import pprint

In [7]:
doc = load_file(os.path.join(DATASHEET_DIR, "MPC5668xRM.pdf"))
content = doc.elements

In [85]:
# Filter out header and footer elements
header_box = BoundingBox(0, 1000, 740, 1000)
footer_box = BoundingBox(0, 1000, 0, 80)
for page in range(1, doc.number_of_pages):
    content -= content.filter_partially_within_bounding_box(header_box, page)
    content -= content.filter_partially_within_bounding_box(footer_box, page)

In [103]:
# Extract module base addresses
mod_base_addr_before = content.filter_by_text_contains("Table A-1. Module Base Addresses")[0]
mod_base_addr_after = content.filter_by_text_contains("Table A-2. MPC5668x System Memory Map")[0]

mod_base_addr_table_elements = content.between(mod_base_addr_before, mod_base_addr_after).filter_by_fonts("IOCJHP+Helvetica-Bold,9.0", "IOCJJB+Helvetica,9.0")
mod_base_addr_table_data = mod_base_addr_table_elements.filter_by_font("IOCJJB+Helvetica,9.0")

t = tables.extract_simple_table(mod_base_addr_table_data, as_text=True)
modules = [(d[0], int(d[1].replace("_", ""), 16), d[2].split("Page ")[1]) for d in t]

for m in modules:
    print(f"Module '{m[0]}': Base: {hex(m[1])}, Page: {m[2]})")

Module 'Program/Data Flash': Base: 0x3fff, Page: A-3)
Module 'Flash Emulation Mapping': Base: 0x1000000, Page: A-3)
Module 'SRAM': Base: 0x40000000, Page: A-3)
Module 'MLB_DIM Configuration': Base: 0xc3f84000, Page: A-8)
Module 'I2C_C': Base: 0xc3f88000, Page: A-11)
Module 'I2C_D': Base: 0xc3f8c000, Page: A-11)
Module 'DSPI_C': Base: 0xc3f90000, Page: A-12)
Module 'DSPI_D': Base: 0xc3f94000, Page: A-13)
Module 'eSCI_J': Base: 0xc3fa0000, Page: A-14)
Module 'eSCI_K': Base: 0xc3fa4000, Page: A-14)
Module 'eSCI_L': Base: 0xc3fa8000, Page: A-15)
Module 'eSCI_M': Base: 0xc3fac000, Page: A-16)
Module 'FlexRay': Base: 0xc3fdc000, Page: A-16)
Module 'AXBS': Base: 0xfff04000, Page: A-38)
Module 'Sema4': Base: 0xfff10000, Page: A-39)
Module 'MPU': Base: 0xfff14000, Page: A-40)
Module 'SWT': Base: 0xfff38000, Page: A-42)
Module 'STM': Base: 0xfff3c000, Page: A-42)
Module 'ECSM': Base: 0xfff40000, Page: A-43)
Module 'eDMA': Base: 0xfff44000, Page: A-43)
Module 'INTC': Base: 0xfff48000, Page: A-46)

In [ ]:
# Extract register map
mod_base_addr_before = content.filter_by_text_contains("Table A-4. MPC5668x Detailed Register Map")[0]
mod_base_addr_after = content.filter_by_text_contains("In this column, R/W indicates a read/write register")[0]

mod_base_addr_table_elements = content.between(mod_base_addr_before, mod_base_addr_after).filter_by_fonts("IOCJHP+Helvetica-Bold,9.0", "IOCJJB+Helvetica,9.0")
mod_base_addr_table_data = mod_base_addr_table_elements.filter_by_font("IOCJJB+Helvetica,9.0")

# for e in mod_base_addr_table_data:
#     print(e.text())

t = tables.extract_table(
        mod_base_addr_table_data,
        as_text=True,
        fix_element_in_multiple_rows=True,
        fix_element_in_multiple_cols=True
    )

In [ ]:
pprint(t)